## Setup
- Create the People model to use for our demo table
- Connect to the demo sqlite db and create a session we can use for querying 

In [1]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class People(Base):
    """Characters mentioned and their role in the puzzle."""

    __tablename__ = "people"

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False, unique=True)
    role = Column(String, nullable=True)

engine = create_engine("sqlite:///temp_test.db", echo=True)
conn = engine.connect()
session = sessionmaker(bind=engine)()
Base.metadata.create_all(engine)




2024-03-25 17:19:09,176 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-25 17:19:09,176 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("people")
2024-03-25 17:19:09,177 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 17:19:09,177 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("people")
2024-03-25 17:19:09,178 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-25 17:19:09,178 INFO sqlalchemy.engine.Engine 
CREATE TABLE people (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	role VARCHAR, 
	PRIMARY KEY (id), 
	UNIQUE (name)
)


2024-03-25 17:19:09,178 INFO sqlalchemy.engine.Engine [no key 0.00023s] ()
2024-03-25 17:19:09,179 INFO sqlalchemy.engine.Engine COMMIT


## Add records to the People table
- Before we query the table, we'll add 2 records for Sprinkles and Sparkles.

In [2]:
# Add records to the People table
c1 = People(id=1, name="Sprinkles", role="Test")
c2 = People(id=2, name="Sparkles", role="Test")
session.add(c1)
session.add(c2)
session.commit()

2024-03-25 17:19:09,184 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-25 17:19:09,186 INFO sqlalchemy.engine.Engine INSERT INTO people (id, name, role) VALUES (?, ?, ?)
2024-03-25 17:19:09,187 INFO sqlalchemy.engine.Engine [generated in 0.00080s] [(1, 'Sprinkles', 'Test'), (2, 'Sparkles', 'Test')]
2024-03-25 17:19:09,188 INFO sqlalchemy.engine.Engine COMMIT


## Create data to query for
- Now we create some data that we want to query for. This could be coming in from a user updating some information that the application needs to use to retrieve the correct record. 
- This example is assuming that the incoming data will have keys that can map meaningfully to the model being used to handle interactions with the database. In this example, all the keys in the *data* dictionary are Column names in the People model. If the data were to have some additional keys that are not present in the People model, that's fine. It would be problematic to have keys that match model Columns accidentally, though.

In [3]:
# Create some data to query for
data = {"id": 2, "name": "Sparkles", "role": "Test", "extra": True}

## Dynamically build query filters
- It's important the the incoming data to query for has keys that map to Columns in the data model in use.
- This approach allows the data model or incoming data request to use any keys and Columns as long as they are in sync with each other.
- An approach to building a list of query filters that is often seen is something like:
    ```
    data = {'id': 5, 'name': 'SKC'}
    filters = [(People.id == data.get('id')), (People.name == data.get('name'))]
    result = session.query(People).filters(*filters)
    ``` 
    And that's a fine way to do it, but this approach is general to any combination of model and object with some keys that align with the model enough to use to query.

In [4]:

model = People
filters = []
# Iterate over data and for each key and value 
for k, v in data.items():
    # Get the column from the model or None
    column = getattr(model, k, None)
    # If the model has a matching column and the value is truthy, 
    # then append the column == value statement to the query filters
    # This will add 3 statements to the filters list, which will be joined with AND
    if column and v:
        filters.append((getattr(column, "__eq__")(v)))

## Check the attributes of the filters

In [5]:

[x.__dict__ for x in filters]

[{'_orig': (272545802, 272544853),
  '_propagate_attrs': immutabledict({'compile_state_plugin': 'orm', 'plugin_subject': <Mapper at 0x108991af0; People>}),
  'left': Column('id', Integer(), table=<people>, primary_key=True, nullable=False),
  'right': BindParameter('%(4360717648 id)s', 2, type_=Integer()),
  'operator': <function _operator.eq(a, b, /)>,
  'type': Boolean(),
  'negate': <function _operator.ne(a, b, /)>,
  '_is_implicitly_boolean': True,
  'modifiers': {}},
 {'_orig': (272544838, 272544928),
  '_propagate_attrs': immutabledict({'compile_state_plugin': 'orm', 'plugin_subject': <Mapper at 0x108991af0; People>}),
  'left': Column('name', String(), table=<people>, nullable=False),
  'right': BindParameter('%(4360718848 name)s', 'Sparkles', type_=String()),
  'operator': <function _operator.eq(a, b, /)>,
  'type': Boolean(),
  'negate': <function _operator.ne(a, b, /)>,
  '_is_implicitly_boolean': True,
  'modifiers': {}},
 {'_orig': (272544826, 272545000),
  '_propagate_attr

## Query using the filters list built by the for loop above

In [6]:

result = session.query(model).filter(*filters).one_or_none()

2024-03-25 17:19:09,221 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-25 17:19:09,229 INFO sqlalchemy.engine.Engine SELECT people.id AS people_id, people.name AS people_name, people.role AS people_role 
FROM people 
WHERE people.id = ? AND people.name = ? AND people.role = ?
2024-03-25 17:19:09,233 INFO sqlalchemy.engine.Engine [generated in 0.00470s] (2, 'Sparkles', 'Test')


## Check that it returned the correct record

In [7]:

result.name

'Sparkles'

## Clean up: remove db file

In [8]:

from pathlib import Path

path = Path("temp_test.db")

if path.exists():
    path.unlink()
